# DS-SF-38 | Unit Project | 3 | Machine Learning Modeling and Executive Summary | Starter Code

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Project 1 and 2.  You will summarize and present your findings and the methods you used.

In [19]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn import linear_model

/Users/andrewyu/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [24]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'dataset-ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether an applicant was admitted.

In [25]:
# TODO
df.prestige.value_counts()

2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64

## Part B.  Feature Engineering

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [27]:
# TODO
cs = pd.get_dummies(df.prestige)
cs.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
cs

,prestige_1,prestige_2,prestige_3,prestige_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
395,0,1,0,0
396,0,0,1,0
397,0,1,0,0
398,0,1,0,0


> ### Question 3.  How many of these binary variables do we need for modeling?

Answer: TODO

4, since there are 4 classifiers

> ### Question 4.  Why are we doing this?



Answer: We want to take categorical values and perform "binarization" so that we can include it as a feature to train the model


> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [28]:
# TODO
# Add columns https://stackoverflow.com/questions/23208745/python-pandas-add-dummy-columns-to-the-original-dataframe
# Drop column http://erikrood.com/Python_References/dropping_rows_cols_pandas.html
df = pd.concat([df, cs], axis=1);
# df.drop(['prestige'], axis=1)
df

,admit,gre,gpa,prestige,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1
...,...,...,...,...,...,...,...,...
395,0,620.0,4.00,2.0,0,1,0,0
396,0,560.0,3.04,3.0,0,0,1,0
397,0,460.0,2.63,2.0,0,1,0,0
398,0,700.0,3.65,2.0,0,1,0,0


In [29]:
df.prestige_1

0      0
1      0
2      1
3      0
4      0
      ..
395    0
396    0
397    0
398    0
399    0
Name: prestige_1, Length: 397, dtype: uint8

## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether an applicant was admitted.

In [30]:
# TODO
# https://chrisalbon.com/python/pandas_crosstabs.html
pd.crosstab(df.admit, df.prestige_1)

prestige_1,0,1
admit,,
0,243,28
1,93,33


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [31]:
# TODO
# Is doing division really supposed to be that hard in python?? https://stackoverflow.com/questions/6781898/how-to-print-the-result-of-a-division-in-ironpython-python-net
print float(33)/float(33+28)

0.540983606557


> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [32]:
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


In [33]:
# TODO
print float(33)/float(33+28) 
print float(53)/float(53+95)
print float(28)/float(28+93)
print float(12)/float(12+55)

0.540983606557
0.358108108108
0.231404958678
0.179104477612


> ### Question 9.  Finally, what's the odds ratio?

In [34]:
# TODO

# @TODO COME BACK TO THIS

> ### Question 10.  Write this finding in a sentence.

Answer: TODO

@TODO COME BACK TO THIS

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentence.

Answer: TODO
@TODO COME BACK TO THIS

## Part D. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model predicting admission into UCLA using `gre`, `gpa`, and the `prestige` of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [35]:
# TODO
X = df[['gre', 'gpa', 'prestige', 'prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']]
c = df.admit
sm.Logit(c, sm.add_constant(X)).fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 8


> ### Question 13.  Print the model's summary results.

In [36]:
# TODO
sm.Logit(c, sm.add_constant(X)).fit().summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 8


/Users/andrewyu/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:1029: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
/Users/andrewyu/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/andrewyu/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/andrewyu/anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 23 Oct 2017   Pseudo R-squ.:                 0.08166
Time:                        21:59:20   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5979   1.18e+07  -1.36e-07      1.000    -2.3e+07     2.3e+07
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
prestige      -1.1090        nan        nan        nan         nan         nan
prestige_1    -1.1699    1.3e+07  -8.99e-08      1.000   -2.55e+07    2.55e+07
prestige_2    -0.7411   1.33e+07  -5.59e-08      1.000    -2.6e+07     2.6e+07
prestige_3    -0.2906   1.25e+07  -2.32e-08      1.000   -2.46e+07    2.46e+07
prestige_4     0.6037   1.07e+07   5.66e-08      1.000   -2.09e+07    2.09e+07
==============================================================================
"""

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [ ]:
# TODO

> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

Answer: TODO

> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer: TODO

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [37]:
# TODO

# @TODO COME BACK TO THIS

Answer: TODO

## Part E. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [38]:
# TODO
model = linear_model.LogisticRegression().fit(X, c)

print model.intercept_
print zip(X.columns, model.coef_[0])

[-0.87964467]
[('gre', 0.0018890017536052387), ('gpa', 0.38460026261179991), ('prestige', -0.84166497031813392), ('prestige_1', -0.52262047211800244), ('prestige_2', -0.44424110243614251), ('prestige_3', -0.22057010724216058), ('prestige_4', 0.30778700709966367)]


> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [ ]:
# TODO

Answer: TODO

> ### Question 20.  Again, assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [ ]:
# TODO

Answer: TODO

## Part F.  Executive Summary

> ## Question 21.  Introduction
>
> Write a problem statement for this project.

Answer: TODO

> ## Question 22.  Dataset
>
> Write up a description of your data and any cleaning that was completed.

Answer: TODO

> ## Question 23.  Demo
>
> Provide a table that explains the data by admission status.

Answer: TODO

> ## Question 24.  Methods
>
> Write up the methods used in your analysis.

Answer: TODO

> ## Question 25.  Results
>
> Write up your results.

Answer: TODO

> ## Question 26.  Visuals
>
> Provide a table or visualization of these results.

Answer: TODO

> ## Question 27.  Discussion
>
> Write up your discussion and future steps.

Answer: TODO